In [ ]:
pip install openai

In [ ]:
pip install git+https://github.com/openai/whisper.git 

In [ ]:
%conda install -c conda-forge ffmpeg

In [ ]:
!conda --version

In [ ]:
ffmpeg -version

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [2]:
import httpx
from openai import OpenAI
api_key=os.getenv("OPENAI_WHISPER_API_KEY")
client=OpenAI(api_key=api_key,http_client=httpx.Client())

In [5]:

def transcribe(name):
# Open the audio file
    with open(f"audio-12262024/{name}", "rb") as audio_file:
        # Transcribe the audio file
        result = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )

    # Specify the file name
    file_name = f"{name[:-3]}-12262024txt"

    # Open the file in write mode ('w')

    with open(file_name, "w") as file:
        # Write your text to the file
        file.write(result.text)

    print(f"Text written to {file_name} successfully.")


In [ ]:
transcribe("11.mp3")

In [ ]:
for n in range(11):
    name = f"0{n+1}.mp3"
    if n >= 10:
        name = f"{n+1}.mp3"
    
    print(name)
    transcribe(name)


In [ ]:
import whisper
import os
print(os.getcwd())

model = whisper.load_model("tiny.en")
os.chdir(os.getcwd())
print(os.getcwd())
file_path = r".\audio\presto.wav"
result = model.transcribe(file_path)
print(result["text"])

In [ ]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
file_path = r".\audio\test.mp3"
audio = whisper.load_audio(file_path)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

In [ ]:
def rewrite_text(client, text, model="o1", temperature=0.7):
    """
    Rewrite a given text using OpenAI API.

    Parameters:
        api_key (str): Your OpenAI API key.
        text (str): The text to rewrite.
        model (str): The OpenAI model to use (default is "gpt-4").
        temperature (float): Sampling temperature (default is 0.7).

    Returns:
        str: The rewritten text.
    """

    try:
        messages=[
            {
                "role": "system",
                "content": "rewrite the text provided by the user in an informal and professional manner."
            }, 
            {
                "role": "user",
                "content": text
            }
        ]
        completion = client.chat.completions.create(
        model=os.getenv("LLM_MODEL_NAME"),
        response_format={ "type": "json_object" },
        messages=messages
    )
    
        return completion.choices[0].message.content
    except client.error.OpenAIError as e:
        return f"An error occurred: {e}"

In [ ]:
text = "this is a sample text. this is a story about a king and a queen."
rewrite_text(client, text)
             

In [ ]:

# Function to generate images
def generate_lost_and_found_images(prompt, num_images):
    images = []
    for _ in range(num_images):
        response = client.images.generate(
            prompt=prompt,
            n=1,  # Number of images per request
            model="dall-e-3",
            size="1024x1024",
            quality="standard"         
        )
        images.append(response.data[0].url)
    return images

# Prompt for lost and found box
prompt = (
    "A plain, unlabelled box placed on a clean surface in a well-lit room. Box contains the One of Each of the following items: "
    "Old keys, Chargers and cables, Small electronics, Manuals and instructions, Disposable gloves, Velcro strips, Bungee cords"
    "Ornaments (e.g., Christmas baubles), String lights (NOT hanging, NOT lit), Holiday figurines, Candle holders"
    "Paintbrushes, Hot glue sticks, Ribbons and twine, Scissors, Measuring tools"
    "Batteries, Lightbulbs, Electrical tape, Duct tape, Extension cords, Super glue"
    "Tennis balls, Golf tees, Hand grips or gloves, Bike repair tools or patches"
    "Plastic containers, Plastic bags, Plastic bottles, Plastic cups, Plastic utensils"
    "Old books, Old magazines, Old newspapers, Old CDs, Old DVDs"
    "Old clothes, Old shoes, Old hats, Old belts, Old wallets"
    "Plant seeds, Gardening gloves, Plant labels, Twine and ties, Small pruning shears"
    "Spare fuses, Spare light bulbs, Spare batteries, Spare chargers, Spare extension cords"
    "screws, nails, bolts, and nuts, Washers and fasteners, Screwdrivers and wrenches, Tape measures, Sandpaper, Small power tool accessories (drill bits, saw blades)"
    "The scene emphasizes the casual mix of items, no more than 10 items, object are arranged in a way that is realistic and natural, but not cluttered."
)

# Generate images
num_images = 1  # Adjust the number of images as needed
image_urls = generate_lost_and_found_images(prompt, num_images)

# Print image URLs
for idx, url in enumerate(image_urls):
    print(f"Image {idx + 1}: {url}")


In [ ]:
import requests
import os
import uuid

def save_images(image_urls, folder="lost_and_found_images"):
    os.makedirs(folder, exist_ok=True)
    for idx, url in enumerate(image_urls):
        response = requests.get(url)
        with open(f"{folder}/image_{idx + 1}_{str(uuid.uuid4())}.png", "wb") as f:
            f.write(response.content)
        print(f"Saved image_{idx + 1}.png")

save_images(image_urls)
